Welcome to quickstart notebook of EfficientNetV2 Keras package.

We will go over some basic concepts, like 
1. Installation.
2. Download data + fine tune.
3. Convert to TFLite.
4. Convert to ONNX.

Execute the cell below to check if we are using a GPU:

In [1]:
!nvidia-smi

Thu Aug  5 15:25:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Installation

Run below cell to install the module:

In [2]:
!pip install -q git+https://github.com/sebastian-sz/efficientnet-v2-keras@main

In [3]:
import os

import tensorflow as tf
from efficientnet_v2 import EfficientNetV2S


print(tf.__version__)

2.5.0


### Download example dataset

In this section we are going to download example dataset.

In [4]:
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  83.7M      0  0:00:02  0:00:02 --:--:-- 83.7M


Remove the License file so it doesn't mess up directory structure:

In [5]:
!rm flower_photos/LICENSE.txt

Preview Class names:

In [6]:
!ls flower_photos

daisy  dandelion  roses  sunflowers  tulips


### Load the data:

In [7]:
DATA_PATH = "./flower_photos"
BATCH_SIZE = 32
TARGET_SIZE = (384, 384)  # S variant expects images in shape (384, 384)


def preprocess_data(images, labels):
    images = (images - 128.00) / 128.00
    return images, labels


def augment_data(images, labels):
    return tf.image.random_flip_left_right(images), labels

In [8]:
# Create tf.data.dataset objects:

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATA_PATH,
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    label_mode="categorical",
    seed=1234,
    validation_split=0.2,
    subset="training"
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATA_PATH,
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    label_mode="categorical",
    seed=1234,
    validation_split=0.2,
    subset="validation"
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [9]:
# Apply preprocessing and augmentation:

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=AUTOTUNE).map(augment_data, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [10]:
# Sanity check our dataset shapes:

for image_batch, label_batch in train_dataset.take(1):
    print(image_batch.shape)
    print(label_batch.shape)

(32, 384, 384, 3)
(32, 5)


### Train (extract features)

Let us fine tune EfficientV2 S variant.

In [11]:
def build_model(num_classes=5):
    base_model = EfficientNetV2S(
        input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3),
        include_top=False,
        pooling="avg",
        weights="imagenet++"  # Let's use pretrained on imagenet 21k and fine tuned on 1k weight variant.
    )

    base_model.trainable=False

    return tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation=None)
    ])

In [12]:
model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

82468864/82461544 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetv2-s (Functional (None, 1280)              20331360  
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 20,337,765
Trainable params: 6,405
Non-trainable params: 20,331,360
_________________________________________________________________


In [13]:
model.fit(
    train_dataset,
    epochs=5,
    validation_data=val_dataset,
)

Epoch 1/5
92/92 [==============================] - 85s 480ms/step - loss: 0.5869 - accuracy: 0.8018 - val_loss: 0.1982 - val_accuracy: 0.9441
Epoch 2/5
92/92 [==============================] - 40s 423ms/step - loss: 0.1833 - accuracy: 0.9421 - val_loss: 0.1480 - val_accuracy: 0.9564
Epoch 3/5
92/92 [==============================] - 40s 432ms/step - loss: 0.1327 - accuracy: 0.9581 - val_loss: 0.1317 - val_accuracy: 0.9564
Epoch 4/5
92/92 [==============================] - 40s 433ms/step - loss: 0.1081 - accuracy: 0.9670 - val_loss: 0.1282 - val_accuracy: 0.9564
Epoch 5/5
92/92 [==============================] - 41s 440ms/step - loss: 0.0851 - accuracy: 0.9758 - val_loss: 0.1160 - val_accuracy: 0.9619


### Convert TFLite

We can convert the modified model to Tensorflow Lite:

In [14]:
# Convert
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save
with open("efficientnet_lite.tflite", "wb") as file:
  file.write(tflite_model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmphf93ejg9/assets


INFO:tensorflow:Assets written to: /tmp/tmphf93ejg9/assets


In [15]:
!ls *.tflite

efficientnet_lite.tflite


### Convert onnx

We can also convert this model to ONNX via `tf2onnx` package:

In [16]:
!pip install tf2onnx~=1.8.4

     |████████████████████████████████| 370 kB 7.4 MB/s 
     |████████████████████████████████| 12.3 MB 60.1 MB/s 


In [17]:
# Save the model in TF's Saved Model format:

In [18]:
model.save("my_saved_model/")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: my_saved_model/assets


INFO:tensorflow:Assets written to: my_saved_model/assets


In [19]:
# Convert:
!python -m tf2onnx.convert \
  --saved-model my_saved_model/ \
  --output efficientnet_lite.onnx

2021-08-05 15:32:48.572785: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-08-05 15:32:49.998243: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-05 15:32:50.003909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-05 15:32:50.004392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBan

In [20]:
!ls *.onnx

efficientnet_lite.onnx
